# 周转率

## 安装软件包

In [ ]:
import sys

In [ ]:
!{sys.executable} -m pip install -r requirements.txt

In [ ]:
import cvxpy as cvx
import numpy as np
import pandas as pd
import time
import os
import quiz_helper
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (14, 8)

### 数据包

In [ ]:
import os
import quiz_helper
from zipline.data import bundles

In [ ]:
os.environ['ZIPLINE_ROOT'] = os.path.join(os.getcwd(), '..', '..','data','module_4_quizzes_eod')
ingest_func = bundles.csvdir.csvdir_equities(['daily'], quiz_helper.EOD_BUNDLE_NAME)
bundles.register(quiz_helper.EOD_BUNDLE_NAME, ingest_func)
print('Data Registered')

### 构建管道引擎

In [ ]:
from zipline.pipeline import Pipeline
from zipline.pipeline.factors import AverageDollarVolume
from zipline.utils.calendars import get_calendar

universe = AverageDollarVolume(window_length=120).top(500) 
trading_calendar = get_calendar('NYSE') 
bundle_data = bundles.load(quiz_helper.EOD_BUNDLE_NAME)
engine = quiz_helper.build_pipeline_engine(bundle_data, trading_calendar)

### 查看数据

构建管道引擎后，我们获取时段结束时股票池中的股票。我们将使用这些 ticker 生成风险模型的收益率数据。

In [ ]:
universe_end_date = pd.Timestamp('2016-01-05', tz='UTC')

universe_tickers = engine\
    .run_pipeline(
        Pipeline(screen=universe),
        universe_end_date,
        universe_end_date)\
    .index.get_level_values(1)\
    .values.tolist()
    
universe_tickers

# 获取收益率数据

In [ ]:
from zipline.data.data_portal import DataPortal

data_portal = DataPortal(
    bundle_data.asset_finder,
    trading_calendar=trading_calendar,
    first_trading_day=bundle_data.equity_daily_bar_reader.first_trading_day,
    equity_minute_reader=None,
    equity_daily_reader=bundle_data.equity_daily_bar_reader,
    adjustment_reader=bundle_data.adjustment_reader)

## 获取股价数据的辅助函数

In [ ]:
def get_pricing(data_portal, trading_calendar, assets, start_date, end_date, field='close'):
    end_dt = pd.Timestamp(end_date.strftime('%Y-%m-%d'), tz='UTC', offset='C')
    start_dt = pd.Timestamp(start_date.strftime('%Y-%m-%d'), tz='UTC', offset='C')

    end_loc = trading_calendar.closes.index.get_loc(end_dt)
    start_loc = trading_calendar.closes.index.get_loc(start_dt)

    return data_portal.get_history_window(
        assets=assets,
        end_dt=end_dt,
        bar_count=end_loc - start_loc,
        frequency='1d',
        field=field,
        data_frequency='daily')

## 将股价数据放入 dataframe 中

In [ ]:
returns_df = \
    get_pricing(
        data_portal,
        trading_calendar,
        universe_tickers,
        universe_end_date - pd.DateOffset(years=5),
        universe_end_date)\
    .pct_change()[1:].fillna(0) #convert prices into returns

returns_df

## 行业数据辅助函数

我们将为你创建一个对象，它会针对每支股票定义一个行业。行业由整数表示。我们继承了 Classifier 类。[Classifier 文档](https://www.quantopian.com/posts/pipeline-classifiers-are-here)以及 [Classifier 的源代码](https://github.com/quantopian/zipline/blob/master/zipline/pipeline/classifiers/classifier.py)

In [ ]:
from zipline.pipeline.classifiers import Classifier
from zipline.utils.numpy_utils import int64_dtype
class Sector(Classifier):
    dtype = int64_dtype
    window_length = 0
    inputs = ()
    missing_value = -1

    def __init__(self):
        self.data = np.load('../../data/project_4_sector/data.npy')

    def _compute(self, arrays, dates, assets, mask):
        return np.where(
            mask,
            self.data[assets],
            self.missing_value,
        )

In [ ]:
sector = Sector()

## 我们将使用 2 年的数据计算因子

**注意：**2 年前的日期是休市日期。管道软件包不会处理开始或结束日期是休市的日期。为了解决这个问题，我们再往回推 2 天，这时候市场没有休市。

In [ ]:
factor_start_date = universe_end_date - pd.DateOffset(years=2, days=2)
factor_start_date

## 创建平滑的动量因子

In [ ]:
from zipline.pipeline.factors import Returns
from zipline.pipeline.factors import SimpleMovingAverage


# create a pipeline called p
p = Pipeline(screen=universe)
# create a factor of one year returns, deman by sector, then rank
factor = (
    Returns(window_length=252, mask=universe).
    demean(groupby=Sector()). #we use the custom Sector class that we reviewed earlier
    rank().
    zscore()
)


# Use this factor as input into SimpleMovingAverage, with a window length of 5
# Also rank and zscore (don't need to de-mean by sector, s)
factor_smoothed = (
    SimpleMovingAverage(inputs=[factor], window_length=5).
    rank().
    zscore()
)

# add the unsmoothed factor to the pipeline
p.add(factor, 'Momentum_Factor')
# add the smoothed factor to the pipeline too
p.add(factor_smoothed, 'Smoothed_Momentum_Factor')

## 可视化管道

注意，如果图片在 notebook 中加载很慢，请右击并在单独的标签页中查看图片。

In [ ]:
p.show_graph(format='png')

## 运行管道并查看因子数据

In [ ]:
df = engine.run_pipeline(p, factor_start_date, universe_end_date)

In [ ]:
df.head()

## 评估因子

我们将介绍评估 alpha 因子可以用到的一些工具。我们将使用 [alphalens 库](https://github.com/quantopian/alphalens)


## 导入 alphalens

In [ ]:
import alphalens as al

## 获取股价数据

注意，我们已经获取股价数据并转换为收益率，然后根据收益率计算因子。我们将再次获取股价数据，但是不会转换为收益率。这是因为我们将使用 alphalens 函数，这些函数的输入必须是股价，不是收益率。

## 定义资产列表

为了确保我们获取的是具有因子值的股票的股价，我们将获取资产列表，该列表可能是原始股票池的子集

In [ ]:
# get list of stocks in our portfolio (tickers that identify each stock)
assets = df.index.levels[1].values.tolist()
print(f"stock universe number of stocks {len(universe_tickers)}, and number of stocks for which we have factor values {len(assets)}")

In [ ]:
factor_start_date

In [ ]:
pricing = get_pricing(
        data_portal,
        trading_calendar,
        assets, #notice that we used assets instead of universe_tickers; in this example, they're the same
        factor_start_date, # notice we're using the same start and end dates for when we calculated the factor
        universe_end_date)

In [ ]:
factor_names = df.columns
print(f"The factor names are {factor_names}")

# Use a dictionary to store each dataframe, one for each factor and its associated forward returns
factor_data = {}
for factor_name in factor_names:
    print("Formatting factor data for: " + factor_name)
    # Get clean factor and forward returns for each factor
    # Choose single period returns (daily returns)
    factor_data[factor_name] = al.utils.get_clean_factor_and_forward_returns(
        factor=df[factor_name],
        prices=pricing,
        periods=[1])

## 周转率分析

与其他因子相比，好的因子不会产生太高的交易成本。为何某些因子会产生更高的交易成本？如果因子要求我们不断再平衡投资组合，每天都需要买卖股票，那么与要求每季度才交易一次的因子相比，这种因子的成本更高。我们可以查看因子排名（每天都按照因子分数排序，然后排名为 1、2、3...N），看看因子排名每天的变化。例如假设投资组合有 3 支股票，排名很多天都没有变化（例如股票 A 始终排在第三名，股票 B 排在第一名，股票 C 排在第二名），那么我们无需在这些天展开交易，使投资组合与 alpha 因子保持一致。

我们可以根据排名随时间推移的自相关性衡量周转率。在量化金融领域，我们将这种自相关性称为“因子排名自相关系数”，简称 FRA。Alphalens 提供了函数 [alphalens.performance.factor_rank_autocorrelation](https://quantopian.github.io/alphalens/alphalens.html?highlight=factor_rank_autocorrelation#alphalens.performance.factor_rank_autocorrelation)

In [ ]:
    factor_rank_autocorrelation(factor_data, period=1):
    
    factor_data: 
      Use the dataframe that is returned from our call to alphalens.utils.get_clean_factor_and_forward_returns

    period:
        Period over which to calculate the turnover.  Keep the default of 1 

## 小测验 1
在调用 factor_rank_autocorrelation 函数并传入因子数据后，看看出现的错误消息。该函数需要什么数据类型？

In [ ]:
factor_names = df.columns
ls_fra = []

for i, factor_name in enumerate(factor_names):
    print("Calculating the FRA for: " + factor_name)
    
    #TODO: look at the error generated from this line of code
    fra = al.performance.factor_rank_autocorrelation(factor_data[factor_name]).to_frame()
    fra.columns = [factor_name]
    ls_fra.append(fra)

df_ls_fra = pd.concat(ls_fra, axis=1)

## 答案 1


## 将日期时间转换为整型

在将因子数据传入 factor_rank_autocorrelation 函数中之前，我们需要使用 unix 将日期时间转换成整型。

In [ ]:
unixt_factor_data = {}
for factor_name in factor_names:
    unixt_index_data = [(x.timestamp(), y) for x, y in factor_data[factor_name].index.values]
    unixt_factor_data[factor_name] = factor_data[factor_name].set_index(pd.MultiIndex.from_tuples(unixt_index_data, names=['date', 'asset']))

## 小测验 2： 
计算因子排名自相关系数

使用将日期时间索引转换成整型时用到的数据

In [ ]:
factor_names = df.columns
ls_fra = []

for i, factor_name in enumerate(factor_names):
    print("Calculating the FRA for: " + factor_name)
    
    #TODO: calculate factor rank autocorrelation
    fra = # ...
    fra.columns = [factor_name]
    ls_fra.append(fra)

df_ls_fra = pd.concat(ls_fra, axis=1)

## 查看输出的 FRA

In [ ]:
df_ls_fra.plot(title="Factor Rank Autocorrelation");

## 小测验 3
未平滑因子和平滑因子的因子排名自相关系数有何区别？这对潜在交易成本来说有何影响？

## 答案 3

## 解答 notebook
[解答 notebook](turnover_solution.ipynb)